In [2]:
import requests
import json
import pandas as pd
import numpy as np
from scipy import stats
from statistics import mean

In [1]:
from secret import *

In [4]:
nasdaq = pd.read_csv('nasdaq_screener.csv')

In [6]:
# Creating dataframe
data = pd.DataFrame()

# Adding data from the nasdaq csv file to the pandas dataframe
for row in nasdaq.index:
    data = data.append(pd.Series(
        [ 
            nasdaq.loc[row][0],
            nasdaq.loc[row][1],
            nasdaq.loc[row][5],
            nasdaq.loc[row][6],
            nasdaq.loc[row][10]
        ],index=['Symbol','Name','Market Cap','Country','Industry']
        ), ignore_index=True, sort=True)

# reordering the columns
columns = ['Symbol','Name','Market Cap','Country','Industry']
data = data.reindex(columns=columns)

In [8]:
# getting rid of symbols with market cap under 1,000,000
for row in data.index:
    market_cap = data.loc[row][2]
    if market_cap < 1000000: 
        data.drop(row, inplace=True)
        
# Defining function to split up list of quotes to lists of 100 because the api has a limit per request
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n] 
        
data.reset_index(drop=True, inplace=True)

# Using the chunks function to break up the symbols into lists of 100 
symbol_groups = list(chunks(data['Symbol'], 100))

# for each list of 100 stocks make the list into a string of 100 symbols seperated by commas and put the strings into a list because of api requirements
symbol_strings = []
for index in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[index]))

In [81]:
# creating new dataframe to store api data 
api_data = pd.DataFrame()
api_data_cols = ['Symbol', 'Price', 'One-Year Price Return','Three-Month Price Return','One-Month Price Return']

In [82]:
# making a batch api call for each list of 100 stocks to get all the data then adding data to pandas dataframe
for symbol_string in symbol_strings:
    iex_data = requests.get(f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol_string}&token={IEX_API_TOKEN}').json()
    for symbol in symbol_string.split(','):
        try:
            api_data = api_data.append(pd.Series(
                [
                    symbol,
                    iex_data[symbol]['quote']['latestPrice'],
                    iex_data[symbol]['advanced-stats']['year1ChangePercent'],
                    iex_data[symbol]['advanced-stats']['month3ChangePercent'],
                    iex_data[symbol]['advanced-stats']['month1ChangePercent']
                ],index = api_data_cols),ignore_index = True)
        except Exception:
            api_data = api_data.append(pd.Series(
                    [
                        np.NAN,
                        np.NAN,
                        np.NAN,
                        np.NAN,
                        np.NAN
                    ],index = api_data_cols),ignore_index=True)
            

In [87]:
# droping the columns with missing data then reindexing the dataframe

api_data.dropna(inplace=True)
api_data = api_data.reindex(columns=api_data_cols)

In [88]:
api_data

,Symbol,Price,One-Year Price Return,Three-Month Price Return,One-Month Price Return
0,AACG,1.260,-0.582711,-0.298856,-0.163241
1,AACI,10.030,0.010326,0.003451,0.002054
2,AADI,16.620,12.773664,-0.170596,-0.172884
3,AAL,19.894,-0.045516,0.23128,0.212794
4,AAME,3.300,-0.201809,0.394604,0.140838
...,...,...,...,...,...
3749,ZVO,0.731,-0.811662,-0.401359,-0.174917
3750,ZWRK,10.023,0.006018,0.009067,0.004779
3751,ZY,1.946,-0.979718,-0.66559,-0.44858
3752,ZYNE,1.500,-0.665511,-0.407,-0.303484


In [91]:
# for each price return period find the percentile each stock is in compared to the other stocks in the list 
time_periods = ['One-Year','Three-Month','One-Month']
for row in api_data.index:
    for time_period in time_periods:
        api_data.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(
            api_data[f'{time_period} Price Return'], api_data.loc[row, f'{time_period} Price Return'])/100

In [93]:
# finding the mean of each stocks price return percentiles
for row in api_data.index:
    momentum_score = []
    for time_period in time_periods:
        momentum_score.append(api_data.loc[row, f'{time_period} Return Percentile'])
    api_data.loc[row, 'Momentum Score'] = mean(momentum_score)

In [98]:
# sorting the dataframe so that the highest momentum scores are on top
api_data.sort_values(by='Momentum Score', ascending=False, inplace=True)
api_data.reset_index(drop=True, inplace=True)

In [102]:
# taking the top 500 stocks and putting them into a seperate dataframe
stocks_to_buy = api_data[:500][['Symbol', 'Price']]

In [104]:
# exporting list of stocks to csv file
stocks_to_buy.to_csv('stocks_to_buy.csv', index=False)